In [1]:
import time as t
import datetime as dt
import pandas as pd
import requests
import numpy as np
from dotenv import load_dotenv
import os
import sys
sys.path.insert(1, '../')
from utils.common import *


load_dotenv('../.env')
apikey = os.getenv('bi_api_key')


@retry(max_attempts=10, retry_delay=1)
def get_price_data(apikey, token, time_from, time_to):
    url = "https://public-api.birdeye.so/defi/ohlcv"
    headers = {"x-chain": "solana", "X-API-KEY": apikey}
    params = {
        "address": token,
        "type": '1m',
        "time_from": time_from,
        "time_to": time_to
    }
    response = requests.get(url, headers=headers, params=params)
    prices_df = pd.DataFrame(response.json()['data']['items'])
    prices_df['address'] = token
    return prices_df

In [2]:
with SQLiteDB('../dbs/calls.db') as conn:
    query = "SELECT * FROM calls "  #WHERE launched is NULL
    df = pd.read_sql_query(query, conn)
    df = df.loc[dt.datetime.now() > pd.to_datetime(df['expected_launch_time_ts']) + dt.timedelta(minutes=1)]
df

,id,date,address,expected_launch_time,expected_launch_time_ts,s_mm2,s_ma2,s_fa2,s_q,s_sni,mcap_num,liq_num,launched,launch_time,buy,buy_time,buy_price
0,215618,2024-04-18 14:35:57,CfQSSRk6Md2cnZCzkJu2QjAVua5FzGMCua7CrhQGRURK,In 1 minute,2024-04-18 14:36:57,0,0,0,2,0,9050,18110,1,2024-04-18 21:37:00,None,None,None
1,215619,2024-04-18 14:37:11,GKMRt3xZznGK4wxoBw3SZfMTwVR9LLFTJxbqMCUvMabg,11 seconds ago,2024-04-18 14:37:00,1,0,0,0,2,1390,2500,1,2024-04-18 21:37:00,None,None,None
2,215620,2024-04-18 14:37:58,66Yp2kKFLsEjszAMVttwHFDZRZRyGt9kbKoXnuv99Dx3,36 seconds ago,2024-04-18 14:37:22,1,0,0,0,4,1730,1390,1,2024-04-18 21:42:00,None,None,None
3,215621,2024-04-18 14:38:20,Ecz4mak5jScXrwZkMGJB3mqqpeuJPWPTddJW6Rgmt9Za,27 seconds ago,2024-04-18 14:37:53,0,0,0,0,2,1110,1110,1,2024-04-18 21:38:00,None,None,None
4,215622,2024-04-18 14:39:15,BdSSmAmWAE8F2yhjAPW91Boa4fQfu98EKt3xxgLKQVxn,36 seconds ago,2024-04-18 14:38:39,1,1,0,0,2,40230,80460,1,2024-04-18 21:39:00,None,None,None


In [3]:
def update_launch_date(index, status, time):
    ltime = f'"{time}"' if time else "NULL"
    with SQLiteDB('../dbs/calls.db') as conn:
        update_statement = f"""
        UPDATE calls
        SET launched = {status}, launch_time = {ltime}
        WHERE id = {index}
        """
        #print(update_statement)
        conn.execute(update_statement)

In [4]:
for i,row in df[:1].iterrows():
    print('trying: ',i, row['address'])
    elt = int(dt.datetime.strptime(row['expected_launch_time_ts'],'%Y-%m-%d %H:%M:%S').timestamp())
    tmp_df = get_price_data(apikey, row['address'], elt - 60*60, elt + 60*60*25)
    if not tmp_df.empty:
        launched = True
        launch_time = dt.datetime.fromtimestamp(tmp_df['unixTime'].min())
    else:
        launched = True
        launch_time = None
    update_launch_date(row['id'], launched, launch_time)

trying:  0 CfQSSRk6Md2cnZCzkJu2QjAVua5FzGMCua7CrhQGRURK


In [10]:
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.jobstores.sqlalchemy import SQLAlchemyJobStore
jobstores = {
    'default': SQLAlchemyJobStore(url='sqlite:///../dbs/jobs.sqlite')
}
scheduler = BackgroundScheduler(jobstores=jobstores)

In [11]:
def core_task():
    a = 1
    b = 2
    res = a+b
    #test = get_price_data(apikey, row['address'], int(launch_time.timestamp()), int((launch_time + dt.timedelta(minutes=20)).timestamp()))
    print("Task executed. Result: ", res)

In [14]:
scheduler.add_job(core_task, 'date', run_date=str(launch_time + dt.timedelta(minutes=100080)), id=str(row['id']))

<Job (id=215618 name=core_task)>

In [15]:
scheduler.print_jobs()

Pending jobs:
    your_task (trigger: date[2024-06-27 09:37:00 +07], pending)
    core_task (trigger: date[2024-06-27 09:37:00 +07], pending)
